# P1

## (a)

In [4]:
def kmean(data, K):

    m = len(data[0])
    n = len(data) 

    n_min = [min([data[j][i] for j in range(n)]) for i in range(m)]
    n_max = [max([data[j][i] for j in range(n)]) for i in range(m)]

    center = [[n_min[i] + (n_max[i] - n_min[i]) * random.random() for i in range(m)] for _ in range(K)]

    while True:
        z = [[0] * K for _ in range(n)]
        for i in range(n):
            distances = [sum((data[i][d] - center[k][d]) ** 2 for d in range(m)) ** 0.5 for k in range(K)]
            min_index = distances.index(min(distances))
            z[i][min_index] = 1
        new_center = [[0] * m for _ in range(K)]
        for k in range(K):
            cluster_points = [data[i] for i in range(n) if z[i][k] == 1]
            if cluster_points:
                for d in range(m):
                    new_center[k][d] = sum(point[d] for point in cluster_points) / len(cluster_points)
            else:
                new_center[k] = center[k]
        if all(sum((new_center[k][d] - center[k][d]) ** 2 for d in range(m)) ** 0.5 < 1 for k in range(K)):
            break
        center = new_center

    result = []
    
    for i in range(n):
        distances = [sum((data[i][d] - center[k][d]) ** 2 for d in range(m)) ** 0.5 for k in range(K)]
        cluster_label = distances.index(min(distances))
        result.append(data[i] + [cluster_label])

    return center, result

In [6]:
def meanshift(data, bandwidth=6, threshold=1e-7):
    import random

    N = len(data)
    D = len(data[0])
    label = list(range(N))
    visit = [0] * N
    count = []
    cluster_center = []
    cluster_count = 0

    while len(label) > 0:
        rnd = random.randint(0, len(label) - 1)
        center = data[label[rnd]]
        frequency = [0] * N

        while True:
            distances = [(sum((center[d] - data[i][d]) ** 2 for d in range(D))) for i in range(N)]
            flag = [i for i, dist in enumerate(distances) if dist < bandwidth ** 2]
            
            for idx in flag:
                visit[idx] = 1
                frequency[idx] += 1
            
            new_center = [0] * D
            sum_weight = 0
            
            for idx in flag:
                weight = 2.718 ** (-distances[idx] / (bandwidth ** 2))  # Gaussian kernel
                sum_weight += weight
                
                for d in range(D):
                    new_center[d] += weight * data[idx][d]
                    
            new_center = [x / sum_weight for x in new_center]

            if sum((new_center[d] - center[d]) ** 2 for d in range(D)) ** 0.5 < threshold:
                break
            center = new_center

        merge_with = -1
        
        for i, existing_center in enumerate(cluster_center):
            if sum((center[d] - existing_center[d]) ** 2 for d in range(D)) ** 0.5 < bandwidth / 2:
                merge_with = i
                break
        
        if merge_with == -1:
            cluster_center.append(center)
            count.append(frequency)
            cluster_count += 1
        else:
            cluster_center[merge_with] = [(cluster_center[merge_with][d] + center[d]) / 2 for d in range(D)]
            count[merge_with] = [count[merge_with][i] + frequency[i] for i in range(N)]
            
        label = [i for i in range(N) if visit[i] == 0]

    result = []
    
    for i in range(N):
        max_count = max(count[j][i] for j in range(cluster_count))
        cluster_label = next(j for j in range(cluster_count) if count[j][i] == max_count)
        result.append(data[i] + [cluster_label])

    return cluster_center, result

In [8]:
import random
from math import pi, exp, sqrt

# Helper functions
def gaussian_probability(x, mu, sigma):
    D = len(x)
    det_sigma = 1
    for d in range(D):
        det_sigma *= sigma[d][d]
    inv_sigma = [[1 / sigma[d][d] if d == j else 0 for j in range(D)] for d in range(D)]
    diff = [x[d] - mu[d] for d in range(D)]
    exponent = -0.5 * sum(diff[d] * inv_sigma[d][d] * diff[d] for d in range(D))
    return (1 / ((2 * pi) ** (D / 2) * sqrt(det_sigma))) * exp(exponent)

def init_parameters(data, K):
    N = len(data)
    D = len(data[0])
    random.shuffle(data)
    miu = [data[i] for i in range(K)]
    sigma = [[[1 if i == j else 0 for j in range(D)] for i in range(D)] for _ in range(K)]
    pi = [1 / K] * K
    return miu, sigma, pi

def em_gaussian(data, K, threshold=1e-3):
    N = len(data)
    D = len(data[0])
    miu, sigma, pi_k = init_parameters(data, K)
    prev_log_likelihood = float('-inf')

    while True:
        # E-step
        P = [[pi_k[k] * gaussian_probability(data[i], miu[k], sigma[k]) for k in range(K)] for i in range(N)]
        gamma = [[p / sum(P[i]) for p in P[i]] for i in range(N)]

        # M-step: Update parameters
        N_k = [sum(gamma[i][k] for i in range(N)) for k in range(K)]
        
        for k in range(K):
            miu[k] = [sum(gamma[i][k] * data[i][d] for i in range(N)) / N_k[k] for d in range(D)]
            sigma[k] = [[0] * D for _ in range(D)]
            for i in range(N):
                diff = [data[i][d] - miu[k][d] for d in range(D)]
                for d1 in range(D):
                    for d2 in range(D):
                        sigma[k][d1][d2] += gamma[i][k] * diff[d1] * diff[d2]
                sigma[k] = [[sigma[k][d1][d2] / N_k[k] for d2 in range(D)] for d1 in range(D)]
            pi_k[k] = N_k[k] / N

        log_likelihood = sum(
            log(sum(pi_k[k] * gaussian_probability(data[i], miu[k], sigma[k]) for k in range(K))) for i in range(N)
        )

        if abs(log_likelihood - prev_log_likelihood) < threshold:
            break
        prev_log_likelihood = log_likelihood

    result = []
    
    for i in range(N):
        probabilities = [pi_k[k] * gaussian_probability(data[i], miu[k], sigma[k]) for k in range(K)]
        cluster_label = probabilities.index(max(probabilities))
        result.append(data[i] + [cluster_label])
        
    return miu, result

## (b)

## (c)

By testing several different values ​​on three data sets, some conclusions are drawn.

When the bandwidth h is large, the mean shift algorithm tends to merge more points into the same cluster, resulting in a rougher clustering result. On the contrary, when h is small, the algorithm will identify the differences between data points more carefully, which may produce more and smaller clusters. A larger bandwidth can smooth out some noise, making the algorithm less sensitive to noise. However, an excessively large h may also cause useful information to be smoothed out, resulting in distorted clustering results.

# P2

## (a)